In [1]:
# import necessary libraries for the analysis
import vcf
import pysam
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import itertools
import ast
from natsort import index_natsorted

In [2]:
## Functions used below are imported from another notebook, run before changing directory

%run filtering_for_somatic_SVs_functions.ipynb
%run overlap_and_range_functions.ipynb

In [3]:
### Load filtered somatic small deletions ###

##
somatic_small_del_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_wo_gap/small_dels'

somatic_small_del_df_names = []

os.chdir(somatic_small_del_path)
temp_files = sorted([i for i in os.listdir(somatic_small_del_path) if 'DS' not in i])

for file_name in temp_files:
    
    globals()[file_name[:-4]] = pd.read_csv(file_name)
    somatic_small_del_df_names.append(file_name[:-4])

In [6]:
### Load filtered somatic large SVs ###

## DELs
somatic_large_DEL_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_wo_gap/large_SVs/DEL'
somatic_large_DEL_df_names = []

os.chdir(somatic_large_DEL_path)
temp_files = sorted([i for i in os.listdir(somatic_large_DEL_path) if 'DS' not in i])

for file_name in temp_files:
    
    globals()[file_name[:-4]] = pd.read_csv(file_name)
    somatic_large_DEL_df_names.append(file_name[:-4])

    
## DUPs
somatic_large_DUP_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_wo_gap/large_SVs/DUP'
somatic_large_DUP_df_names = []

os.chdir(somatic_large_DUP_path)
temp_files = sorted([i for i in os.listdir(somatic_large_DUP_path) if 'DS' not in i])

for file_name in temp_files:
    
    globals()[file_name[:-4]] = pd.read_csv(file_name)
    somatic_large_DUP_df_names.append(file_name[:-4])

    
## INVs
somatic_large_INV_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_wo_gap/large_SVs/INV'
somatic_large_INV_df_names = []

os.chdir(somatic_large_INV_path)
temp_files = sorted([i for i in os.listdir(somatic_large_INV_path) if 'DS' not in i])

for file_name in temp_files:
    
    globals()[file_name[:-4]] = pd.read_csv(file_name)
    somatic_large_INV_df_names.append(file_name[:-4])

In [4]:
### Load panels of normal samples ###

##
panel_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/panel'

panel_df_names = []

os.chdir(panel_path)
temp_files = sorted([i for i in os.listdir(panel_path) if 'DS' not in i])

for file_name in temp_files:
    
    globals()[file_name[:-4]] = pd.read_csv(file_name)
    panel_df_names.append(file_name[:-4])

In [5]:
panel_df_names

['large_del_norm_panel',
 'large_dup_norm_panel',
 'large_inv_norm_panel',
 'small_del_norm_panel']

In [ ]:
'''
VARIABLE NAMES

somatic_small_del_df_names

somatic_large_DEL_df_names

somatic_large_DUP_df_names

somatic_large_INV_df_names
'''

In [6]:
def percent_overlap(input_start, input_end, gene_start, gene_end):
    
    o  = min(input_end, gene_end) - max(input_start, gene_start) + 1    
    perc = np.round(o/float(int(gene_end)-int(gene_start)+1)*100, 2)
    
    return perc

In [7]:
### Modified overlap function for this file

def overlap_func(input_chr, input_start, input_end, input_ind, df):
    
    output_df = df[df['CHROM'] == input_chr]
    output_df = output_df[(output_df['END'] >= input_start) & (output_df['POS'] <= input_end)]
    
    if len(output_df) != 0:
        
        output_df['percent_overlap'] = df.apply(lambda row: \
                                                percent_overlap(input_start, input_end, row['POS'], row['END']), axis=1)

        output_df['input_start'] = input_start
        output_df['input_end'] = input_end
        output_df['input_ind'] = input_ind
    
    return output_df

In [8]:
### Modified overlap function 2 for this file

def overlap_func_mod(input_chr, input_start, input_end, input_ind, norm_df):
    
    output_df = norm_df[norm_df['CHROM'] == input_chr]
    output_df = output_df[(output_df['END'] >= input_start) & (output_df['POS'] <= input_end)]
    
    if len(output_df) != 0:
        
        output_df['percent_overlap_wrt_normal'] = norm_df.apply(lambda row: \
                                              percent_overlap(input_start, input_end, row['POS'], row['END']), axis=1)
        
        output_df['percent_overlap_wrt_SV'] = norm_df.apply(lambda row: \
                                                  percent_overlap(row['POS'], row['END'], input_start, input_end), axis=1)

        output_df['input_start'] = input_start
        output_df['input_end'] = input_end
        output_df['input_ind'] = input_ind
    
    return output_df

In [9]:
def keep_repeated(numbers):
    
    # Create a dictionary to store the frequency of each number
    frequency = {}
    
    for num in numbers:
        
        if num in frequency:
            frequency[num] += 1
        
        else:
            frequency[num] = 1
    
    # Filter the numbers that appeared twice or more
    result = [num for num, count in frequency.items() if count >= 2]
    
    return result

In [10]:
test = [1, 2, 2, 3, 3, 3, 4, 1, 5, 6, 7, 7]
keep_repeated(test)

[1, 2, 3, 7]

In [ ]:
### Somatic small deletions

somatic_small_del_wo_highoverlap_df_names = []

for df_name in somatic_small_del_df_names:
    
    temp_df = globals()[df_name].copy()
    
    print('Currently analyzing: ' + df_name)
    
    for index, row in temp_df.iterrows():
        
        temp_overlap_df = overlap_func_mod(row['CHROM'], row['POS'], row['END'], index, small_del_norm_panel)
        
        if len(temp_overlap_df) >= 1:
                       
            drop_index = []

            percent_bool = True
            
            for i, r in temp_overlap_df.iterrows():
                
                # percent_bool evaluates if percent overlap is above threshold, for each overlapping sample SV
                
                if r['percent_overlap_wrt_SV'] >= 50:
                    
                    percent_bool = True
                    
                else:
                    
                    percent_bool = False
                    
                    
                if percent_bool:
                    drop_index.append(index)
                
            drop_index_repeated = keep_repeated(drop_index)
            
            # drop SVs with high overlaps
            if len(drop_index_repeated) >= 1:
                
                for i in drop_index_repeated:
                    
                    temp_df = temp_df.drop(i)
            
                    
    temp_df = temp_df.reset_index(drop=True)  
    temp_new_df_name = df_name + '_wo_highoverlap'
    globals()[temp_new_df_name] = temp_df.copy()

    somatic_small_del_wo_highoverlap_df_names.append(temp_new_df_name)

Currently analyzing: A_RR_GBM809_somatic_dels_panel_wo_gap
Currently analyzing: A_R_GBM607_somatic_dels_panel_wo_gap
Currently analyzing: B_P_GBM593_somatic_dels_panel_wo_gap
Currently analyzing: B_R_GBM898_somatic_dels_panel_wo_gap
Currently analyzing: C_P_GBM577_somatic_dels_panel_wo_gap
Currently analyzing: C_R_GBM625_somatic_dels_panel_wo_gap
Currently analyzing: E_RR_GBM937_somatic_dels_panel_wo_gap
Currently analyzing: E_R_GBM781_somatic_dels_panel_wo_gap
Currently analyzing: F_P_GBM620_somatic_dels_panel_wo_gap
Currently analyzing: F_R_GBM691_somatic_dels_panel_wo_gap
Currently analyzing: G_P_GBM454_somatic_dels_panel_wo_gap
Currently analyzing: G_R_GBM833_somatic_dels_panel_wo_gap
Currently analyzing: H_P_GBM460_somatic_dels_panel_wo_gap
Currently analyzing: H_R_GBM492_somatic_dels_panel_wo_gap
Currently analyzing: I_P_GBM440_somatic_dels_panel_wo_gap
Currently analyzing: I_R_GBM532_somatic_dels_panel_wo_gap
Currently analyzing: J_P_GBM401_somatic_dels_panel_wo_gap
Currently an

In [14]:
### Somatic large deletions

somatic_large_DEL_wo_highoverlap_df_names = []

for df_name in somatic_large_DEL_df_names:
    
    temp_df = globals()[df_name].copy()
    
    print('Currently analyzing: ' + df_name)
    
    for index, row in temp_df.iterrows():
        
        temp_overlap_df = overlap_func_mod(row['CHROM'], row['POS'], row['END'], index, large_del_norm_panel)
        
        if len(temp_overlap_df) >= 1:
                       
            drop_index = []

            temp_svlen = 0
            percent_bool = True
            
            for i, r in temp_overlap_df.iterrows():
                
                # percent_bool evaluates if percent overlap is above threshold, for each overlapping sample SV
                
                if r['percent_overlap_wrt_SV'] >= 50:
                    
                    percent_bool = True
                    
                else:
                    
                    percent_bool = False
                    
                    
                if percent_bool:
                    drop_index.append(index)
                
            drop_index_repeated = keep_repeated(drop_index)
            
            # drop SVs with high overlaps
            if len(drop_index_repeated) >= 1:
                
                for i in drop_index_repeated:
                    
                    temp_df = temp_df.drop(i)
                        
                    
    temp_df = temp_df.reset_index(drop=True)  
    temp_new_df_name = df_name + '_wo_highoverlap'
    globals()[temp_new_df_name] = temp_df.copy()

    somatic_large_DEL_wo_highoverlap_df_names.append(temp_new_df_name)

Currently analyzing: A_RR_GBM809_somatic_SV_DELs_panel_wo_gap
Currently analyzing: A_R_GBM607_somatic_SV_DELs_panel_wo_gap
Currently analyzing: B_P_GBM593_somatic_SV_DELs_panel_wo_gap
Currently analyzing: B_R_GBM898_somatic_SV_DELs_panel_wo_gap
Currently analyzing: C_P_GBM577_somatic_SV_DELs_panel_wo_gap
Currently analyzing: C_R_GBM625_somatic_SV_DELs_panel_wo_gap
Currently analyzing: E_RR_GBM937_somatic_SV_DELs_panel_wo_gap
Currently analyzing: E_R_GBM781_somatic_SV_DELs_panel_wo_gap
Currently analyzing: F_P_GBM620_somatic_SV_DELs_panel_wo_gap
Currently analyzing: F_R_GBM691_somatic_SV_DELs_panel_wo_gap
Currently analyzing: G_P_GBM454_somatic_SV_DELs_panel_wo_gap
Currently analyzing: G_R_GBM833_somatic_SV_DELs_panel_wo_gap
Currently analyzing: H_P_GBM460_somatic_SV_DELs_panel_wo_gap
Currently analyzing: H_R_GBM492_somatic_SV_DELs_panel_wo_gap
Currently analyzing: I_P_GBM440_somatic_SV_DELs_panel_wo_gap
Currently analyzing: I_R_GBM532_somatic_SV_DELs_panel_wo_gap
Currently analyzing: J

In [15]:
### Somatic large duplications

somatic_large_DUP_wo_highoverlap_df_names = []

for df_name in somatic_large_DUP_df_names:
    
    temp_df = globals()[df_name].copy()
    
    print('Currently analyzing: ' + df_name)
    
    for index, row in temp_df.iterrows():
        
        temp_overlap_df = overlap_func_mod(row['CHROM'], row['POS'], row['END'], index, large_dup_norm_panel)
        
        if len(temp_overlap_df) >= 1:
                       
            drop_index = []

            temp_svlen = 0
            percent_bool = True
            
            for i, r in temp_overlap_df.iterrows():
                
                # percent_bool evaluates if percent overlap is above threshold, for each overlapping sample SV
                    
                if r['percent_overlap_wrt_SV'] >= 50:
                    
                    percent_bool = True
                    
                else:
                    
                    percent_bool = False
                    
                    
                if percent_bool:
                    drop_index.append(index)
                
            drop_index_repeated = keep_repeated(drop_index)
            
            # drop SVs with high overlaps
            if len(drop_index_repeated) >= 1:
                
                for i in drop_index_repeated:
                    
                    temp_df = temp_df.drop(i)
                        
                    
    temp_df = temp_df.reset_index(drop=True)  
    temp_new_df_name = df_name + '_wo_highoverlap'
    globals()[temp_new_df_name] = temp_df.copy()

    somatic_large_DUP_wo_highoverlap_df_names.append(temp_new_df_name)

Currently analyzing: A_RR_GBM809_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: A_R_GBM607_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: B_P_GBM593_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: B_R_GBM898_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: C_P_GBM577_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: C_R_GBM625_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: E_RR_GBM937_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: E_R_GBM781_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: F_P_GBM620_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: F_R_GBM691_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: G_P_GBM454_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: G_R_GBM833_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: H_P_GBM460_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: H_R_GBM492_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: I_P_GBM440_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: I_R_GBM532_somatic_SV_DUPs_panel_wo_gap
Currently analyzing: J

In [16]:
### Somatic large inversions

somatic_large_INV_wo_highoverlap_df_names = []

for df_name in somatic_large_INV_df_names:
    
    temp_df = globals()[df_name].copy()
    
    print('Currently analyzing: ' + df_name)
    
    for index, row in temp_df.iterrows():
        
        temp_overlap_df = overlap_func_mod(row['CHROM'], row['POS'], row['END'], index, large_inv_norm_panel)
        
        if len(temp_overlap_df) >= 1:
                       
            drop_index = []

            temp_svlen = 0
            percent_bool = True
            
            for i, r in temp_overlap_df.iterrows():
                
                # percent_bool evaluates if percent overlap is above threshold, for each overlapping sample SV
                    
                if r['percent_overlap_wrt_SV'] >= 50:
                    
                    percent_bool = True
                    
                else:
                    
                    percent_bool = False
                    
                    
                if percent_bool:
                    drop_index.append(index)
                
            drop_index_repeated = keep_repeated(drop_index)
            
            # drop SVs with high overlaps
            if len(drop_index_repeated) >= 1:
                
                for i in drop_index_repeated:
                    
                    temp_df = temp_df.drop(i)
                        
                    
    temp_df = temp_df.reset_index(drop=True)  
    temp_new_df_name = df_name + '_wo_highoverlap'
    globals()[temp_new_df_name] = temp_df.copy()

    somatic_large_INV_wo_highoverlap_df_names.append(temp_new_df_name)

Currently analyzing: A_RR_GBM809_somatic_SV_INVs_panel_wo_gap
Currently analyzing: A_R_GBM607_somatic_SV_INVs_panel_wo_gap
Currently analyzing: B_P_GBM593_somatic_SV_INVs_panel_wo_gap
Currently analyzing: B_R_GBM898_somatic_SV_INVs_panel_wo_gap
Currently analyzing: C_P_GBM577_somatic_SV_INVs_panel_wo_gap
Currently analyzing: C_R_GBM625_somatic_SV_INVs_panel_wo_gap
Currently analyzing: E_RR_GBM937_somatic_SV_INVs_panel_wo_gap
Currently analyzing: E_R_GBM781_somatic_SV_INVs_panel_wo_gap
Currently analyzing: F_P_GBM620_somatic_SV_INVs_panel_wo_gap
Currently analyzing: F_R_GBM691_somatic_SV_INVs_panel_wo_gap
Currently analyzing: G_P_GBM454_somatic_SV_INVs_panel_wo_gap
Currently analyzing: G_R_GBM833_somatic_SV_INVs_panel_wo_gap
Currently analyzing: H_P_GBM460_somatic_SV_INVs_panel_wo_gap
Currently analyzing: H_R_GBM492_somatic_SV_INVs_panel_wo_gap
Currently analyzing: I_P_GBM440_somatic_SV_INVs_panel_wo_gap
Currently analyzing: I_R_GBM532_somatic_SV_INVs_panel_wo_gap
Currently analyzing: J

In [ ]:
### Validation: small deletions

print('SMALL DELETIONS')

for df_name in somatic_small_del_df_names:
    
    temp_df1 = globals()[df_name]
    temp_df2 = globals()[df_name + '_wo_highoverlap']
    
    if len(temp_df1) != len(temp_df2):
        print(df_name[:-26])
        print('Difference = ' + str(len(temp_df1) - len(temp_df2)))
        print('New count = ' + str(len(temp_df2)))

In [17]:
### Validation: large deletions

print('LARGE DELETIONS')

for df_name in somatic_large_DEL_df_names:
    
    temp_df1 = globals()[df_name]
    temp_df2 = globals()[df_name + '_wo_highoverlap']
    
    if len(temp_df1) != len(temp_df2):
        print(df_name[:-29])
        print('Difference = ' + str(len(temp_df1) - len(temp_df2)))
        print('New count = ' + str(len(temp_df2)))

LARGE DELETIONS
A_RR_GBM809
Difference = 28
New count = 9
A_R_GBM607
Difference = 23
New count = 2
B_P_GBM593
Difference = 11
New count = 8
B_R_GBM898
Difference = 7
New count = 6
C_P_GBM577
Difference = 9
New count = 9
C_R_GBM625
Difference = 13
New count = 2
E_RR_GBM937
Difference = 20
New count = 33
E_R_GBM781
Difference = 10
New count = 11
F_P_GBM620
Difference = 21
New count = 12
F_R_GBM691
Difference = 16
New count = 13
G_P_GBM454
Difference = 17
New count = 4
G_R_GBM833
Difference = 14
New count = 5
H_P_GBM460
Difference = 15
New count = 21
H_R_GBM492
Difference = 16
New count = 15
I_P_GBM440
Difference = 24
New count = 34
I_R_GBM532
Difference = 12
New count = 3
J_P_GBM401
Difference = 24
New count = 24
J_RR_GBM551
Difference = 22
New count = 39
J_R_GBM498
Difference = 37
New count = 46
K_P_GBM529
Difference = 12
New count = 16
K_R_GBM832
Difference = 39
New count = 89
L_P_GBM618
Difference = 43
New count = 23
L_R_SMTB152
Difference = 69
New count = 77
M_P_GBM672
Difference = 1

In [18]:
### Validation: large duplications

print('LARGE DUPLICATIONS')

for df_name in somatic_large_DUP_df_names:
    
    temp_df1 = globals()[df_name]
    temp_df2 = globals()[df_name + '_wo_highoverlap']
    
    if len(temp_df1) != len(temp_df2):
        print(df_name[:-29])
        print('Difference = ' + str(len(temp_df1) - len(temp_df2)))
        print('New count = ' + str(len(temp_df2)))

LARGE DUPLICATIONS
A_RR_GBM809
Difference = 18
New count = 62
A_R_GBM607
Difference = 23
New count = 30
B_P_GBM593
Difference = 23
New count = 25
B_R_GBM898
Difference = 20
New count = 29
C_P_GBM577
Difference = 9
New count = 60
C_R_GBM625
Difference = 17
New count = 24
E_RR_GBM937
Difference = 33
New count = 78
E_R_GBM781
Difference = 19
New count = 24
F_P_GBM620
Difference = 21
New count = 21
F_R_GBM691
Difference = 23
New count = 54
G_P_GBM454
Difference = 26
New count = 15
G_R_GBM833
Difference = 19
New count = 19
H_P_GBM460
Difference = 21
New count = 111
H_R_GBM492
Difference = 21
New count = 111
I_P_GBM440
Difference = 25
New count = 20
I_R_GBM532
Difference = 21
New count = 26
J_P_GBM401
Difference = 26
New count = 35
J_RR_GBM551
Difference = 25
New count = 70
J_R_GBM498
Difference = 25
New count = 54
K_P_GBM529
Difference = 10
New count = 31
K_R_GBM832
Difference = 23
New count = 199
L_P_GBM618
Difference = 40
New count = 37
L_R_SMTB152
Difference = 41
New count = 33
M_P_GBM67

In [19]:
### Validation: large inversions

print('LARGE INVERSIONS')

for df_name in somatic_large_INV_df_names:
    
    temp_df1 = globals()[df_name]
    temp_df2 = globals()[df_name + '_wo_highoverlap']
    
    if len(temp_df1) != len(temp_df2):
        print(df_name[:-29])
        print('Difference = ' + str(len(temp_df1) - len(temp_df2)))
        print('New count = ' + str(len(temp_df2)))

LARGE INVERSIONS
A_RR_GBM809
Difference = 1
New count = 2
A_R_GBM607
Difference = 4
New count = 12
B_R_GBM898
Difference = 1
New count = 4
C_R_GBM625
Difference = 2
New count = 12
E_RR_GBM937
Difference = 5
New count = 15
E_R_GBM781
Difference = 3
New count = 14
F_P_GBM620
Difference = 2
New count = 5
F_R_GBM691
Difference = 3
New count = 25
G_P_GBM454
Difference = 2
New count = 6
G_R_GBM833
Difference = 2
New count = 6
H_P_GBM460
Difference = 3
New count = 4
H_R_GBM492
Difference = 4
New count = 3
I_P_GBM440
Difference = 2
New count = 16
I_R_GBM532
Difference = 3
New count = 5
J_RR_GBM551
Difference = 1
New count = 12
J_R_GBM498
Difference = 2
New count = 14
K_R_GBM832
Difference = 2
New count = 7
L_P_GBM618
Difference = 1
New count = 2
L_R_SMTB152
Difference = 2
New count = 9
M_P_GBM672
Difference = 5
New count = 10
M_R_GBM828
Difference = 2
New count = 0
N_P_BT2013110
Difference = 2
New count = 7
N_R_GBM745
Difference = 3
New count = 17
O_P_GBM703
Difference = 1
New count = 9
O_R_SM

In [ ]:
### Save updated somatic small dels ###

somatic_small_del_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_final/small_dels'

os.chdir(somatic_small_del_path)soma

for df_name in somatic_small_del_wo_highoverlap_df_names:
    
    globals()[df_name].to_csv((somatic_small_del_path + '/' + df_name + '.csv'), index=False, sep=',')

In [20]:
### Save updated somatic large SVs ###

somatic_large_sv_path = '/Users/ryanyutian/Desktop/new_panel_TRI_Brain_glioma_sv_processed/somatic_panel_final/large_svs'

    
os.chdir(somatic_large_sv_path)

for df_name in somatic_large_DEL_wo_highoverlap_df_names:
    
    globals()[df_name].to_csv((somatic_large_sv_path + '/DEL/' + df_name + '.csv'), index=False, sep=',')
    
for df_name in somatic_large_DUP_wo_highoverlap_df_names:
    
    globals()[df_name].to_csv((somatic_large_sv_path + '/DUP/' + df_name + '.csv'), index=False, sep=',')

for df_name in somatic_large_INV_wo_highoverlap_df_names:
    
    globals()[df_name].to_csv((somatic_large_sv_path + '/INV/' + df_name + '.csv'), index=False, sep=',')